In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
import torch
import sys
sys.path.append('/home/QFRC/ZQS/Projects/LLaMA-Factory-main/src')
import llmtuner
from llmtuner.data.template import get_template_and_fix_tokenizer
from llmtuner.extras.misc import get_logits_processor
import pandas as pd


In [3]:
# 加载model、config
model_path = "/home/QFRC/ZQS/Projects/LLaMA-Factory-main/script/TF-14B/Merges/TF-14B-80event_cp900"

# 加载微调后的tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
# 加载微调后的模型
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True, device_map="cuda:1")
# Specify hyperparameters for generation
model.generation_config = GenerationConfig.from_pretrained(model_path, trust_remote_code=True)
config=model.generation_config
config_dict=config.to_dict()
#type(config_dict)
config_dict

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

{'max_length': 20,
 'max_new_tokens': 512,
 'min_length': 0,
 'min_new_tokens': None,
 'early_stopping': False,
 'max_time': None,
 'do_sample': True,
 'num_beams': 1,
 'num_beam_groups': 1,
 'penalty_alpha': None,
 'use_cache': True,
 'temperature': 0.85,
 'top_k': 0,
 'top_p': 0.8,
 'typical_p': 1.0,
 'epsilon_cutoff': 0.0,
 'eta_cutoff': 0.0,
 'diversity_penalty': 0.0,
 'repetition_penalty': 1.0,
 'encoder_repetition_penalty': 1.0,
 'length_penalty': 1.0,
 'no_repeat_ngram_size': 0,
 'bad_words_ids': None,
 'force_words_ids': None,
 'renormalize_logits': False,
 'constraints': None,
 'forced_bos_token_id': None,
 'forced_eos_token_id': None,
 'remove_invalid_values': False,
 'exponential_decay_length_penalty': None,
 'suppress_tokens': None,
 'begin_suppress_tokens': None,
 'forced_decoder_ids': None,
 'sequence_bias': None,
 'guidance_scale': None,
 'low_memory': None,
 'num_return_sequences': 1,
 'output_attentions': False,
 'output_hidden_states': False,
 'output_scores': False,


In [16]:
df=pd.read_excel(io='/home/QFRC/ZQS/Projects/Data/阳光电源（21-24）_deldata_process.xlsx')
data=df['total'][:10]
print(data)
print(type(data))
for news in data:
    print(news)
    # print(type(news))
    
  

0                 中国银河维持阳光电源推荐评级 预计2024年净利润同比增长26.31%。
1    东莞证券给予阳光电源买入评级。东莞证券04月24日发布研报称，给予阳光电源（300274.S...
2                 东莞证券维持阳光电源买入评级 预计2024年净利润同比增长10.06%。
3            开源证券维持阳光电源买入(Buy)评级 预计2024年净利润同比增长20.94%。
4                 西部证券维持阳光电源买入评级 预计2024年净利润同比增长13.81%。
5    阳光电源2023年净利润大涨163% 2024年一季度再增39%。4月22日晚间，阳光电源发...
6    东吴证券给予阳光电源买入评级，2023年报&2024一季报点评：2024Q1盈利略超预期 目...
7    阳光电源(300274.SZ)：2024年前一季度净利润为20.96亿元 同比增长39.05...
8                              东吴证券维持阳光电源买入评级 目标价127元。
9                 民生证券维持阳光电源推荐评级 预计2024年净利润同比增长15.03%。
Name: total, dtype: object
<class 'pandas.core.series.Series'>
中国银河维持阳光电源推荐评级 预计2024年净利润同比增长26.31%。
东莞证券给予阳光电源买入评级。东莞证券04月24日发布研报称，给予阳光电源（300274.SZ，最新价：95.91元）买入评级。评级理由主要包括：1）公司业绩保持增长，盈利能力同比改善；2）光伏逆变器等电力电子转换设备出货量大增；3）储能业务保持较快增长，发力布局氢能新技术，持续深化海外业务。风险提示：下游需求不及预期风险；竞争加剧带来的毛利率降低风险；汇率波动风险。, AI点评：阳光电源近一个月获得3份券商研报关注，买入2家，平均目标价为127元，与最新价95.91元相比，高31.09元，目标均价涨幅32.42%。
东莞证券维持阳光电源买入评级 预计2024年净利润同比增长10.06%。
开源证券维持阳光电源买入(Buy)评级 预计2024年净利润同比增长20.94%。
西部证券维持阳光电

In [17]:
  
def prepare_gen_kwargs(input_text):
    """
    准备生成模型输入参数的函数

    参数:
    - ✅instruction: 字符串，指令描述。
    - input_text: 字符串，输入文本。
    - ✅tokenizer: 已加载的tokenizer。
    - ✅model: 已加载到设备上的模型。
    - ✅config_dict: 字典，包含生成配置的参数。

    返回:
    - gen_kwargs: 字典，包含用于模型生成的参数。
    """
    # 获取模板并修正tokenizer
    template1 = get_template_and_fix_tokenizer("default", tokenizer)
    
    instruction='假如你是一名资深的金融分析师，1、提取该文本中你认为对股市及上市公司股价影响较大的重大事件，并以方便构建金融知识图谱的三元组（实体—动作—实体）或二元组（实体—动作）的形式列出，其中实体均为上市公司。2、情感有五类：非常积极，积极，中性，消极，非常消极，需要特别注意的是，非常积极指的是可能导致股价急速上涨的重大积极事件，比如重大资产重组，重大政策之类等，同理，非常消极指的是可能导致股价急速下跌的重大消极事件，比如涉嫌财务造假等，对该文本进行情感分析，并指出属于五类中的哪一类。'
    # 合并指令和输入文本
    inputs = instruction + input_text
    
    # 使用模板和tokenizer处理输入
    prompt, _ = template1.encode_oneturn(tokenizer, inputs, "")
    
    prompt_length = len(prompt)
    # 转换为模型的输入格式
    input_ids = torch.tensor([prompt], device=model.device)
    
    # 准备生成参数
    gen_kwargs = dict(
        inputs=input_ids,
        generation_config=GenerationConfig(**config_dict),
        logits_processor=get_logits_processor()
    )
    
    return gen_kwargs, prompt_length




In [18]:
from openpyxl.utils import get_column_letter
import openpyxl
import json
import pandas as pd

# 创建一个新的Excel工作簿
wb = openpyxl.Workbook()
sheet = wb.active
titles = ['source', 'time', 'sentiment', 'events', 'total', 'entity1', 'relationship', 'entity2']
sheet.append(titles)

# 读取数据
df = pd.read_excel(io='/home/QFRC/ZQS/Projects/Data/阳光电源（21-24）_deldata_process.xlsx')
news = df['total1']  # series
time = df['renew']

current_row = 2  # 从第二行开始填充数据

for i in range(len(news)):
    print(i)
    sheet['A' + str(current_row)] = news.iloc[i]
    sheet['B' + str(current_row)] = time.iloc[i]
    
    # 模型推理
    gen_kwargs, prompt_length = prepare_gen_kwargs(news.iloc[i])
    generate_output = model.generate(**gen_kwargs)
    response_ids = generate_output[:, prompt_length:]
    response = tokenizer.batch_decode(response_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    print(response)
    generate_str = response[0]
    
    try:
        generate_dict = json.loads(generate_str)
        
        sheet['C' + str(current_row)] = generate_dict.get('sentiment', 0)
        
        # 确认 'events' 是一个字典
        if isinstance(generate_dict.get('events'), dict):
            for event, details in generate_dict['events'].items():
                # 在每个事件数据行插入source和time
                sheet['D' + str(current_row)] = event
                
                # 从第五列(E列)开始填充事件的详细信息
                col = 5
                if isinstance(details, dict):  # 确认 details 是字典
                    for key, value in details.items():
                        column_letter = get_column_letter(col)  # 转换成对应的列字母
                        sheet[column_letter + str(current_row)] = value
                        col += 1  # 移动到下一列

                current_row += 1  # 处理完一个事件后换行到下一行
        else:
            sheet['D' + str(current_row)] = "非有效事件格式"
            current_row += 1

    except json.JSONDecodeError:
        # JSON格式不正确，跳到下一行
        print(f"第 {i} 条新闻的响应不是有效的JSON格式。")
        sheet['C' + str(current_row)] = "非有效json格式"
        current_row += 1  # 跳到下一行
        continue  # 继续下一条新闻的处理

wb.save('/home/QFRC/ZQS/Projects/Graph/data/三元组/阳光电源_三元组1.xlsx')

0
['{"events": {"event1": {"total": "阳光电源预计2024年净利润同比增长26.31%", "entity1": "阳光电源", "relationship": "2024年净利润同比增长26.31%", "entity2": "预计"}}, "sentiment": "中性"}']
1
['{"events": {"event1": {"total": "阳光电源获得东莞证券买入评级", "entity1": "阳光电源", "relationship": "获得买入评级", "entity2": "东莞证券"}, "event2": {"total": "阳光电源光伏逆变器等电力电子转换设备出货量大增", "entity1": "阳光电源", "relationship": "出货量大增", "entity2": "光伏逆变器等电力电子转换设备"}, "event3": {"total": "阳光电源持续深化海外业务", "entity1": "阳光电源", "relationship": "持续深化", "entity2": "海外业务"}}, "sentiment": "积极"}']
2
['{"events": {"event1": {"total": "阳光电源预计2024年净利润同比增长10.06%", "entity1": "阳光电源", "relationship": "2024年净利润同比增长10.06%", "entity2": "预计"}}, "sentiment": "中性"}']
3
['{"events": {"event1": {"total": "阳光电源拟发行全球存托凭证并在瑞士证券交易所上市", "entity1": "阳光电源", "relationship": "发行", "entity2": "全球存托凭证", "地点": "瑞士证券交易所"}, "event2": {"total": "阳光电源2024年净利润同比增长20.94%", "entity1": "阳光电源", "关系": "2024年净利润同比增长20.94%"}}, "sentiment": "积极"}']
4
['{"events": {"event1": {"total": "阳光电源预计2024年净利润同比增长13

: 

In [5]:
from openpyxl.utils import get_column_letter
import openpyxl
import json
import pandas as pd

# 创建一个新的Excel工作簿
wb = openpyxl.Workbook()
sheet = wb.active
titles = ['source', 'time', 'sentiment', 'events', 'total', 'entity1', 'relationship', 'entity2']
sheet.append(titles)

# 读取数据
df = pd.read_excel(io='/home/QFRC/ZQS/Projects/Data/比亚迪（21-24）_deldata3.xlsx')
news = df['total1']  # series
time = df['renew']

current_row = 2  # 从第二行开始填充数据

for i in range(len(news)):
    print(i)
    sheet['A' + str(current_row)] = news.iloc[i]
    sheet['B' + str(current_row)] = time.iloc[i]
    
    # 模型推理
    gen_kwargs, prompt_length = prepare_gen_kwargs(news.iloc[i])
    generate_output = model.generate(**gen_kwargs)
    response_ids = generate_output[:, prompt_length:]
    response = tokenizer.batch_decode(response_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    print(response)
    generate_str = response[0]
    
    try:
        generate_dict = json.loads(generate_str)
        
        sheet['C' + str(current_row)] = generate_dict.get('sentiment', 0)
        
        # 确认 'events' 是一个字典
        if isinstance(generate_dict.get('events'), dict):
            for event, details in generate_dict['events'].items():
                # 在每个事件数据行插入source和time
                sheet['D' + str(current_row)] = event
                
                # 从第五列(E列)开始填充事件的详细信息
                col = 5
                if isinstance(details, dict):  # 确认 details 是字典
                    for key, value in details.items():
                        column_letter = get_column_letter(col)  # 转换成对应的列字母
                        sheet[column_letter + str(current_row)] = value
                        col += 1  # 移动到下一列

                current_row += 1  # 处理完一个事件后换行到下一行
        else:
            sheet['D' + str(current_row)] = "非有效事件格式"
            current_row += 1

    except json.JSONDecodeError:
        # JSON格式不正确，跳到下一行
        print(f"第 {i} 条新闻的响应不是有效的JSON格式。")
        sheet['C' + str(current_row)] = "非有效json格式"
        current_row += 1  # 跳到下一行
        continue  # 继续下一条新闻的处理

wb.save('/home/QFRC/ZQS/Projects/Graph/data/三元组/比亚迪_三元组3.xlsx')

0
06/18/2024 22:40:40 - INFO - llmtuner.data.template - Add eos token: <|endoftext|>
06/18/2024 22:40:40 - INFO - llmtuner.data.template - Add pad token: <|endoftext|>
['{"events": {"event1": {"total": "比亚迪发布全新高端品牌", "entity1": "比亚迪", "relationship": "发布", "entity2": "全新高端品牌"}, "event2": {"total": "比亚迪股价上涨", "entity1": "比亚迪", "relationship": "股价上涨", "entity2": "高端品牌"}}, "sentiment": "积极"}']
1
['{"events": {"event1": {"total": "比亚迪计划于今年底推高端品牌", "entity1": "比亚迪", "relationship": "推高端品牌"}, "event2": {"total": "比亚迪股价可能上涨", "entity1": "比亚迪", "relationship": "推高端品牌"}}, "sentiment": "积极"}']
2
['{"events": {"event1": {"total": "长城汽车销量下滑", "entity1": "长城汽车", "relationship": "销量下滑"}, "event2": {"total": "哈弗H6销量被超越", "entity1": "哈弗H6", "relationship": "销量被超越"}, "event3": {"total": "比亚迪超越南北大众", "entity1": "比亚迪", "relationship": "超越", "entity2": "南北大众"}, "event4": {"total": "比亚迪全面开花", "entity1": "比亚迪", "relationship": "全面开花"}}, "sentiment": "积极"}']
3
['{"events": {"event1": {"total": "比亚迪计划推出百万元级别高

In [ ]:
from openpyxl.utils import get_column_letter
import openpyxl
import json

# 创建一个新的Excel工作簿
wb = openpyxl.Workbook()
sheet = wb.active
titles = ['source', 'time', 'sentiment', 'events', 'total', 'entity1', 'relationship', 'entity2']
sheet.append(titles)

df=pd.read_excel(io='/home/QFRC/ZQS/Projects/Data/宁德时代（21-24）_deldata.xlsx')
news=df['total'] # series
time=df['renew']

current_row = 2  # 从第二行开始填充数据

for i in range(len(news)):
    print(i)
    sheet['A' + str(current_row)] = news.iloc[i]
    sheet['B' + str(current_row)] = time.iloc[i]
       
    gen_kwargs, prompt_length = prepare_gen_kwargs(news.iloc[i])
    generate_output = model.generate(**gen_kwargs)
    response_ids = generate_output[:, prompt_length:]
    response = tokenizer.batch_decode(
            response_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
        )
    print(response)
    generate1=response
    generate_str=generate1[0]
    # 
    try:
        generate_dict=json.loads(generate_str)
        
        sheet['C' + str(current_row)] = generate_dict.get('sentiment',0)
        
        for event, details in generate_dict['events'].items():
        # 在每个事件数据行插入source和time
            sheet['D' + str(current_row)] = event
            
            # 从第五列(E列)开始填充事件的详细信息
            col = 5
            for key, value in details.items():
                column_letter = get_column_letter(col)  # 转换成对应的列字母
                sheet[column_letter + str(current_row)] = value
                col += 1  # 移动到下一列

            current_row += 1  # 处理完一个事件后换行到下一行
            
    except json.JSONDecodeError:
        # JSON格式不正确，跳到下一行
        print(f"第 {i} 条新闻的响应不是有效的JSON格式。")
        sheet['C' + str(current_row)] = "非有效json格式"
        current_row += 1  # 跳到下一行
        continue  # 继续下一条新闻的处理

wb.save('/home/QFRC/ZQS/Projects/Graph/data/三元组/宁德时代_三元组1.xlsx')

# for news in data:
#     gen_kwargs, prompt_length = prepare_gen_kwargs(news)
#     generate_output = model.generate(**gen_kwargs)
#     response_ids = generate_output[:, prompt_length:]
#     response = tokenizer.batch_decode(
#             response_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
#         )
    
#     sources =df['total'][:1].iloc[10] # series
#     times=df['renew'][:10].iloc[0] # series
    
    

In [37]:
text=['{"events": {"event1": {"total": "阳光电源与华能新能源签署战略合作协议", "entity1": "阳光电源", "relationship": "签署战略合作协议", "entity2": "华能新能源"}, "event2": {"total": "阳光电源参与共建“绿电+储能”生态平台", "entity1": "阳光电源", "relationship": "参与共建", "entity2": "“绿电+储能”生态平台"}}, "sentiment": "中性"}']
text=text[0]
text=json.loads(text)

In [22]:
import openpyxl
from openpyxl.utils import get_column_letter

# 创建一个新的Excel工作簿
wb = openpyxl.Workbook()
sheet = wb.active

sources =df['total'][:1].iloc[0] # series
times=df['renew'][:1].iloc[0] # series

for source in sources:    
    source=source

for time in times:
    time=time

# print(source)    
# print(type(time))

# 设置Excel标题行
titles = ['source', 'time', 'sentiment', 'events', 'total', 'entity1', 'relationship', 'entity2']
sheet.append(titles)

# 开始填充数据
current_row = 2  # 从第二行开始填充数据
sheet['A' + str(current_row)] = sources
sheet['B' + str(current_row)] = times
sheet['C' + str(current_row)] = generate_dict['sentiment']
for event, details in generate_dict['events'].items():
    # 在每个事件数据行插入source和time
    sheet['D' + str(current_row)] = event
    
    # 从第五列(E列)开始填充事件的详细信息
    col = 5
    for key, value in details.items():
        column_letter = get_column_letter(col)  # 转换成对应的列字母
        sheet[column_letter + str(current_row)] = value
        col += 1  # 移动到下一列

    current_row += 1  # 处理完一个事件后换行到下一行

# 保存工作簿到一个新的文件
wb.save('阳光电源_三元组.xlsx')

In [ ]:
# 剥离提取字典键值对


import json

# 转字典
generate1=response
print(generate1)
print(type(generate1))
generate_str=generate1[0]
print(generate_str)
print(type(generate_str))
generate_dict=json.loads(generate_str)
print(generate_dict)
print(type(generate_dict))
print(generate_dict)

# 查找某值
event1_total=generate_dict['events']['event1']['total']
print(event1_total)


events_data = generate_dict['events'] # 事件组
sentiment = generate_dict['sentiment'] # 情感值


for event_seq, detail in events_data.items():  
    for key, value in detail.items():
       
        print(event_seq, value)
    




# for event, details in generate_dict['events'].items():
#     events_data[event] = details

# print(generate_dict['events'].items())
# print(events_data)
# print(type(events_data))
# 可操作性不强
# def print_dict(d, indent=0): # indent 参数用来控制打印时的缩进，使输出更易于阅读。
#     for key, value in d.items():
#         print('  ' * indent + str(key) + ":") # 提取一级键；键名前面的空格数量由当前的 indent 级别决定（每级缩进两个空格），这样在输出中就能清晰地看到嵌套的层次结构。
#         if isinstance(value, dict):
#             print_dict(value, indent+1)  # 如果值是字典，递归调用自己   
#         else:
#             print('  ' * (indent+1) + str(value))
            
# print_dict(generate_dict)


In [3]:

template1=get_template_and_fix_tokenizer("default",tokenizer)
instruction='假如你是一名资深的金融分析师，1、提取该文本中你认为对股市及上市公司股价影响较大的重大事件，并以方便构建金融知识图谱的三元组（实体—动作—实体）或二元组（实体—动作）的形式列出，其中实体均为上市公司。2、情感有五类：非常积极，积极，中性，消极，非常消极，需要特别注意的是，非常积极指的是可能导致股价急速上涨的重大积极事件，比如重大资产重组，重大政策之类等，同理，非常消极指的是可能导致股价急速下跌的重大消极事件，比如涉嫌财务造假等，对该文本进行情感分析，并指出属于五类中的哪一类。'
input ='新闻：公告精选：宁德时代拟135亿投建新型锂电池制造基地项目 中核钛白上调钛白粉售价。 ,  , , 【热点】, , 宁德时代：拟135亿元投建新型锂电池生产制造基地(宜春)项目, 宁德时代(300750)9月13日晚间公告，公司拟在江西省宜春市投资建设宁德时代新型锂电池生产制造基地(宜春)项目，项目总投资不超过135亿元。项目建设期为预计自开工建设起不超过30个月，项目规划用地面积约1300亩。, , 中核钛白：上调公司各型号钛白粉销售价格, 中核钛白(002145)9月13日晚间公告，根据近期钛白粉市场行情及原材料价格上涨等因素，公司价格委员会研究决定从即日起，在现有钛白粉销售价格基础上，全面上调公司各型号钛白粉销售价格。其中：国内客户销售价格上调1000元人民币/吨，国际客户出口价格上调150美元/吨。, , 中青宝：公司尚处于元宇宙初步探索阶段触及概念相对较浅, 5个交易日实现3个涨停的中青宝(300052)9月13日晚间披露股价异动公告：元宇宙是一个巨大的概念和模式，公司尚处于初步探索阶段，触及概念相对较浅，对应产品尚在研发中，新游戏上线时间和地区亦受到诸多因素影响。项目实施进度、能否达到公司预计的效果、能否满足未来市场的需求，均存在较大的不确定性。即便以上条件皆满足，公司依然不排除相关产品与元宇宙存在较大差距的可能性。, , 氯碱化工：特种树脂提升改造项目、氯乙烯项目于近期相继开车, 氯碱化工(600618)9月13日晚间披露股票交易异动公告，经公司自查，公司目前生产经营正常，公司市场环境或行业政策未发生重大变化，内部生产经营秩序正常，公司投资建设的特种树脂提升改造项目、20万吨/年氯乙烯项目已于近期相继开车，无其他应披露而未披露的重大信息。, , 塔牌集团：近期水泥价格持续上涨对业绩影响不大, 塔牌集团(002233)9月13日晚间披露股票交易异动公告，公司已披露的经营情况、内外部经营环境未发生重大变化。公司关注到近期媒体报道广西、广东能耗双控及遏制“两高”项目盲目发展，水泥价格持续上涨，近期水泥价格持续上涨会对公司经营业绩会产生一定影响，但影响程度不大。, , 中辰股份：公司暂未开展特高压相关业务, 中辰股份(300933)9月13日晚间披露股票交易异动公告，公司注意到近期特高压板块相关股票涨幅较大、关注度较高，特高压是指±800千伏及以上的直流电和1000千伏及以上交流电的电压等级。公司主要产品为110kV及以下电力电缆、750kV及以下裸导线、电气装备用电线电缆以及电缆附件，暂未开展特高压相关业务。, , 京投发展股价异动提示风险：主营业务仍以房地产开发为主, 京投发展(600683)9月13日晚间披露股票交易风险提示性公告，目前公司主营业务仍为房地产开发、经营及租赁，未发生变化。公司股票近日涨幅较大，最近十个交易日内出现七次涨停，公司短期内股票交易波动较大，敬请广大投资者注意二级市场交易风险。, , 【业绩】, , 双星新材：前三季度净利预增110%-115%, 双星新材(002585)9月13日晚间公告，公司预计前三季度盈利9.46亿元-9.68亿元，同比增长110%-115%。报告期内，五大板块新材料市场持续增长，光学材料、新能源材料增长势头旺盛。新签订单较上年同期大幅增长，并超过去年同期订单规模，创造历史新高。随着销售量的增加，公司加快新产线安装、调试、投产，新增产线产品销售使效益进一步提升。, , 伊戈尔：前三季度净利预增459%—484%, 伊戈尔(002922)9月13日晚间公告，公司预计前三季度盈利1.77亿元—1.85亿元，同比增长458.79%—484.03%。报告期内，公司销售规模进一步扩大，能源产品、照明产品同比均取得了稳定的增长。其中，受益于光伏行业的快速发展，应用于光伏行业的高频磁性器件及光伏升压变压器同比大幅增长；同时，因报告期照明行业景气度高，公司照明电源产品取得了较为稳定的增长。, , 罗牛山：8月生猪销售收入环比增12.40%, 罗牛山(000735)9月13日晚间公告，公司2021年8月销售生猪2.89万头，销售收入5705.30万元，环比变动分别为10.31%、12.40%，同比变动分别为43.61%、-33.23%。1-8月，公司累计销售生猪23.27万头，同比增长155.68%；累计销售收入58,520.08万元，同比增长43.75%。, , 【增减持】, , 吉林敖东：控股股东拟以1.5亿元-3亿元增持公司股份, 吉林敖东(000623)9月13日晚间公告，控股股东金诚公司拟以1.5亿元-3亿元增持股份，增持价格不超过23元/股，增持计划的实施期限：自公告披露之日起6个月内。, 立昂微：股东宁波利时及三名董监高拟减持公司股份, 立昂微(605358)9月13日晚间公告，持股比例6.82%的股东宁波利时，以及公司董事、副总经理、财务总监、董事会秘书吴能云、副总经理咸春雷、监事会主席任德孝拟分别减持不超过3%、0.089%、0.025%、0.006%公司股份。, 蓝盾光电：股东拟减持公司不超7.15%股份, 蓝盾光电(300862)9月13日晚间公告，持股7.1525%的股东宁波庐熙股权投资合伙企业(有限合伙)计划自15个交易日后的6个月内通过集中竞价交易方式或自3个交易日后的6个月内通过大宗交易方式合计减持公司股份不超过943万股(占公司总股本比例7.1525%)。, 河化股份：股东拟减持不超6%公司股份, 河化股份(000953)9月13日晚间公告，合计持股6.88%的股东王进文、王翠莲拟减持不超过6%的公司股份。, , 【合同中标】, , 江特电机：拟与国轩高科在电池级碳酸锂领域开展合作, 江特电机(002176)9月13日晚间公告，公司9月12日与国轩高科签署战略合作框架协议，拟在电池级碳酸锂领域开展合作，除此之外还可在各自新能源产业链上有优势竞争能力项目进行探讨，比如储能、新能源电机等。此外，公司表示，今年以来，公司股票价格涨幅巨大。虽然公司锂矿资源储量较丰富，但受采选能力受限、储量较大的矿山“探转采”工作周期长以及市场原材料采购紧张等因素影响，公司达成今年碳酸锂产量目标存在不确定性。, 国机汽车：签署1.31亿元经营合同, 国机汽车(600335)9月13日晚间公告，子公司中汽工程与华域汽车车身零件(沈阳)有限公司签署《大东区二厂厂房改扩建建设项目设计施工总承包(EPC)合同》，合同价款1.31亿元。合同的签订将对合同履行期内的公司经营业绩产生积极影响。, 奥特维：签订8478万元硅片分选机采购合同, 奥特维(688516)9月13日晚间公告，公司于当日与弘元新材料(包头)有限公司签订《硅片分选机采购合同》，公司向弘元新材料(包头)有限公司销售硅片分选机设备，合同金额8478万元(含税)。因公司商品平均验收周期为6-9个月，受合同具体交货批次及验收时间的影响，合同履行对2021年业绩影响存在不确定性，将对公司2022年经营业绩产生积极的影响。, 九洲集团：签署生物质热电联产项目合作框架协议, 九洲集团(300040)9月13日晚间公告，近日，公司与国家电投集团黑龙江绿拓新能源有限公司签署了《生物质热电联产项目合作框架协议》。经双方协商，将共同组建股份公司，建立混合所有制的股份制企业，联合开发双方持有的生物质热电联产和综合智慧能源项目，充分发挥央企和民企各自优势，打造出国内生物质综合智慧能源的头部企业，实现合资公司在国内资本市场单独上市的目标。, , 【重大投资】, , 桂东电力：签署风光火储项目投资框架协议书总投资110亿元, 桂东电力(600310)9月13日晚间公告，近日，公司与广西贺州市八步区人民政府经友好协商，就广西桂东电力风光火储蓄能项目建设达成初步投资开发意向，共同签署《广西桂东电力风光火储项目投资框架协议书》。风电项目计划投资约20亿元，抽水蓄能项目计划投资约65亿元，热电联产项目计划投资约5亿元，光伏项目计划投资约20亿元。项目总投资110亿元。, , 国投电力：拟参与投资孟底沟水电站项目总投资347.22亿元, 国投电力(600886)9月13日晚间公告，拟与川投能源共同投资雅砻江流域水电开发有限公司孟底沟水电站。孟底沟水电站动态总投资347.22亿元，其中资本金69.44亿元，占总投资的20%，由国投电力、川投能源按52%、48%的股权比例共同出资。, , 四维图新：拟投资参股国汽智图建设高精度基础地图服务平台, 四维图新(002405)9月13日晚间公告，近日，公司联合工信部国家级创新中心国汽(北京)智能网联汽车研究院有限公司、自然资源部直属企业中国地图出版社等单位共同投资成立国汽智图(北京)科技有限公司。公司以自有资金出资976万元，占比24.0988%。国汽智图定位行业Tier1.5，多方共同建设符合国家数据安全要求，同时满足智能网联汽车行业发展需求的“国家级”关键动态高精度基础地图服务平台。, 永创智能：拟投建年产4万台套包装设备建设项目, 永创智能(603901)9月13日晚间公告，公司于当日与长兴经济技术开发区管理委员会签订《项目投资合作协议》，公司拟通过在长兴经济技术开发区设立全资子公司，投资建设“年产4万台套包装设备建设项目”。项目总投资约5.01亿元(包含流动资金)。项目达产后，预计将增加年销售额约6亿元，对公司未来的经营效益产生积极影响。, 有研新材：拟3.28亿元投建靶材扩产项目, 有研新材(600206)9月13日晚间公告，公司拟投资建设有研亿金新材料有限公司靶材扩产项目，总投资额3.28亿元。该项目包括山东德州新建生产基地和昌平基地升级改造两部分，产品结构和生产工艺进一步优化和改善，靶材产能达到73000块/年。德州基地新建项目建设期12个月，昌平基地提升项目升级改造期17个月，2024年底前达产。, 泰晶科技：拟12亿元投建超小型号低功耗、高端高稳频控元器件生产线, 泰晶科技(603738)9月13日晚间公告，公司于9月11日与重庆市垫江县人民政府签订协议，公司计划在重庆垫江工业园区县城组团建设超小型号低功耗、高端高稳频控元器件生产线，计划投资总金额为12亿元，其中：首期投资4亿元，预计24个月内建成投产，项目总投资5年内完成。, , 长远锂科：子公司拟33.39亿元投建车用锂电池正极材料扩产二期项目, 长远锂科(688779)9月13日晚间公告，公司全资子公司湖南长远锂科新能源有限公司拟投资建设车用锂电池正极材料扩产二期项目，建设4万吨/年正极材料生产线，该项目总投资约为33.39亿元。项目计划2022年1月底开始建设，2023年10月投产，建设期20个月。, , 【股权变动】, , 三聚环保：海淀国投集团子公司收购公司29.48%股份, 三聚环保(300072)9月13日晚间公告，控股股东海淀科技与海淀国投集团全资子公司北京海新致于9月13日签署《股份转让协议》，海淀科技将其持有的公司6.92亿股股份(占公司总股本的29.48%)以6.46元/股的价格通过协议转让的方式转让给北京海新致，合计转让价款为44.72亿元。本次权益变动后，北京海新致将成为公司的控股股东，公司实际控制人仍为北京市海淀区人民政府国有资产监督管理委员会，未发生变更。, , 金花股份：邢博越及其一致行动人成为公司第一大股东, 金花股份(600080)9月13日晚间公告，9月1日至9月13日，股东邢博越及其一致行动人通过二级市场集合竞价方式增持公司股份373.26万股，占公司总股本1%。此次权益变动后，邢博越及其一致行动人持有公司的股份比例由23.35%增加至24.35%，已成为第一大股东。, , 共达电声：实控人拟变更股票明日复牌, 共达电声(002655)9月13日晚间公告，控股股东爱声声学拟将10.28%公司股份以总价款4.63亿元转让给无锡韦感，同时将其剩余4.99%股份对应表决权不可撤销地、独家地委托给无锡韦感使用；若前述股份转让及表决权委托事项最终完成，无锡韦感将成为公司的控股股东。股票明日复牌。, , 【并购重组】, 鄂武商A:拟收购南昌苏宁广场项目总价款33亿元, 鄂武商A(000501)9月13日晚间公告，拟收购南昌苏宁置业持有的南昌市东湖区广场北路以东、省府北二路以南控规SF01-04、SF01-05地块在建商业项目。本次收购项目总价款为33亿元，其中标的项目中资产对价为21.12亿元，公司支付给未来一定期间内总包方等施工方、供应商的价款为11.88亿元。, 泛海控股：武汉金控决定终止对民生证券的收购工作, 泛海控股(000046)9月13日晚间公告，9月10日，公司收到武汉金控发来的函件。武汉金控决定正式终止本次对民生证券的收购工作。公司将继续与其他投资者就民生证券引进战略投资者事宜进行积极接洽。此外，公司亦将继续与武汉金控在武汉金融城建设、金融服务、产业与投资等方面加强合作，争取实现共赢发展目标。, , 【再融资】, 玲珑轮胎：拟定增募资不超25亿元用于全钢子午线轮胎和半钢子午线轮胎生产项目, 玲珑轮胎(601966))9月13日晚间披露非公开发行A股股票预案，募集资金总额预计不超过25亿元，用于长春年产120万条全钢子午线轮胎和300万条半钢子午线轮胎生产项目及补充流动资金。, 沪光股份：拟定增募资不超过7.03亿元, 沪光股份(605333)9月13日晚间公告，拟定增募资不超过7.03亿元，用于昆山泽轩汽车电器有限公司汽车线束部件生产项目和补充流动资金。, , 【其他】, , 上汽集团：耗资4984.7万元首次回购股份, 上汽集团(600104)公司于9月13日以集中竞价交易方式实施了首次回购股份，首次回购股份数量为240.8万股，占公司总股本的比例约0.02%，购买的最高价格为20.80元/股，购买的最低价格为20.61元/股，已支付的资金总额为4984.7万元。, , 天齐锂业：董事会同意公司发行H股股票并在香港联交所上市, 天齐锂业(002466)9月13日晚间公告，公司董事会审议同意公司发行境外上市外资股(H股)股票并申请在香港联合交易所有限公司主板挂牌上市。, 埃斯顿：拟以5000万元至1亿元回购股份, 埃斯顿(002747)9月13日晚间公告，公司拟以集中竞价交易方式，使用不低于5000万元且不超过1亿元回购公司股票，用于员工持股计划或股权激励计划，回购的价格不超过30元/股。, 昂立教育：拟出售韩国CDL公司股票, 昂立教育(600661)9月13日晚间公告，为适应双减政策下公司业务及产品的调整，公司将择机出售公司持有的韩国Chungdahm Learning，INC。(以下简称“CDL公司”)43.2万股流通股股票。截至9月7日，CDL公司的收盘价为29250韩元/股，公司持有的CDL公司股票对应的价值为126.36亿韩元，折合约为7020万元人民币。CDL公司主要从事K12英语和数学课程产品的研发和教学。, 爱康科技：回购股份的价格调整为不超7.5元/股, 爱康科技(002610)9月13日晚间公告，近期公司股价持续上涨，公司股价已经超过此前回购股份方案的回购价格上限。目前公司经营向好，基于对公司未来发展的信心，公司拟继续实施回购股份方案，本次回购股份的价格由不低于人民币2.2元/股(含)且不超过人民币3.6元/股(含)调整为不超过人民币7.5元/股(含)。, 福昕软件：拟1亿-1.5亿元回购股份, 福昕软件(688095)9月13日晚间公告，拟以1亿-1.5亿元回购股份，回购价格不超过238.53元/股。, , 云天化：子公司金新化工临时停产, 云天化(600096)9月13日晚间公告，9月7日，公司所属子公司呼伦贝尔金新化工有限公司发生一名员工在工作现场昏迷后经抢救无效死亡的事件。事件发生后，金新化工及时按照有关规定向相关主管部门报告，并根据要求于9月10日装置按流程停产，目前事件的具体原因正在调查中。金新化工主要产能为尿素80万吨/年。目前，事件的原因尚在调查中，金新化工复产时间及对公司生产经营的影响尚不能确定。'
inputs=instruction+input
prompt,_=template1.encode_oneturn(tokenizer,inputs,"")
prompt_length = len(prompt)
#print(prompt)
input_ids = torch.tensor([prompt], device=model.device)
#print(input_ids)
gen_kwargs = dict(
            inputs=input_ids,
            generation_config=GenerationConfig(**config_dict),
            logits_processor=get_logits_processor()
        )


generate_output = model.generate(**gen_kwargs)
response_ids = generate_output[:, prompt_length:]
response = tokenizer.batch_decode(
            response_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
        )
results = []
print(response)

06/18/2024 08:08:37 - INFO - llmtuner.data.template - Add eos token: <|endoftext|>
06/18/2024 08:08:37 - INFO - llmtuner.data.template - Add pad token: <|endoftext|>
['{"events": {"event1": {"total": "宁德时代拟135亿元投建新型锂电池生产制造基地(宜春)项目", "entity1": "宁德时代", "relationship": "拟135亿元投建新型锂电池生产制造基地", "entity2": "宜春"}, "event2": {"total": "双星新材前三季度净利预增110%-115%", "entity1": "双星新材", "relationship": "前三季度净利预增110%-115%", "entity2": "市场"}, "event3": {"total": "吉林敖东控股股东拟增持1.5亿元-3亿元", "entity1": "吉林敖东", "relationship": "控股股东拟增持1.5亿元-3亿元", "entity2": "市场"}, "event4": {"total": "江特电机与国轩高科在电池级碳酸锂领域开展合作", "entity1": "江特电机", "relationship": "与国轩高科在电池级碳酸锂领域开展合作"}, "event5": {"total": "国投电力参与投资建设雅砻江流域水电开发有限公司孟底沟水电站", "entity1": "国投电力", "relationship": "参与投资建设", "entity2": "孟底沟水电站"}, "event6": {"total": "长远锂科子公司拟33.39亿元投建车用锂电池正极材料扩产二期项目", "entity1": "长远锂科", "relationship": "子公司拟33.39亿元投建车用锂电池正极材料扩产二期项目"}, "event7": {"total": "玲珑轮胎拟定增不超过25亿元用于全钢子午线轮胎和半钢子午线轮胎生产项目", "entity1": "玲珑轮胎", "relationship": "拟定增不超过25亿元用于